In [1]:
import asyncio
from mavsdk import System
from mavsdk.offboard import PositionNedYaw, OffboardError, VelocityNedYaw

drone = System()
await drone.connect(system_address="udp://:14540")

print("Waiting for drone to connect...")
async for state in drone.core.connection_state():
    if state.is_connected:
        print(f"-- Connected to drone!")
        break

async for health in drone.telemetry.health():
    if health.is_global_position_ok and health.is_home_position_ok:
        print("-- Global position state is good enough for flying.")
        break

print("-- Arming")
await drone.action.arm()

print(await drone.offboard.is_active())

print("-- Setting initial setpoint")
startSetpoint = PositionNedYaw(0.0, 0.0, 0.0, 0.0)
await drone.offboard.set_position_ned(startSetpoint)

print("-- Starting offboard")
try:
    await drone.offboard.start()
except OffboardError as error:
    print(f"Starting offboard mode failed with error code:"
            f" {error._result.result}")
    print("-- Disarming")
    await drone.action.disarm()

await asyncio.sleep(10)
await drone.offboard.set_position_ned(PositionNedYaw(2,2,-5,0))
await asyncio.sleep(10)
await drone.offboard.set_position_ned(PositionNedYaw(2,-2,-5,0))
await asyncio.sleep(10)
await drone.offboard.set_position_ned(PositionNedYaw(-2,-2,-5,0))
await asyncio.sleep(10)
await drone.offboard.set_position_ned(PositionNedYaw(-2,2,-5,0))
await asyncio.sleep(10)


Waiting for drone to connect...
-- Connected to drone!
-- Global position state is good enough for flying.
-- Arming
False
-- Setting initial setpoint
-- Starting offboard
